In [ ]:
!pip install transformers accelerate  bitsandbytes langchain_community langchain_huggingface huggingface_hub sentence_transformers faiss-cpu pypdf nltk rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━

In [ ]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [ ]:
from huggingface_hub import login

login()


In [ ]:
model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Configure 4-bit quantization to reduce memory usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the model with quantization settings
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"  # Automatically selects GPU if available
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
tiny_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

Device set to use cuda:0
<ipython-input-6-4f5e4a775e6b>:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  tiny_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
import pandas as pd
# Load the CSV file
df = pd.read_csv('/content/Processed_booking_data.csv')

In [ ]:
from langchain.schema.document import Document
# Convert DataFrame rows into formatted text
documents = []
for ind in df.index:
    text = (
        f"Hotel: {df['hotel'][ind]} | "
        f"Lead Time: {df['lead_time'][ind]} | "
        f"Year: {df['arrival_date_year'][ind]} | "
        f"Month: {df['arrival_date_month'][ind]} | "
        f"Weekend Nights: {df['stays_in_weekend_nights'][ind]} | "
        f"Week Nights: {df['stays_in_week_nights'][ind]} | "
        f"Country: {df['country'][ind]} | "
        f"Reservation Status: {df['reservation_status'][ind]} | "
        f"Reservation Status Date: {df['reservation_status_date'][ind]} | "
        f"Is Canceled: {df['is_canceled'][ind]} | "
        f"Adults: {df['adults'][ind]} | "
        f"Children: {df['children'][ind]} | "
        f"Babies: {df['babies'][ind]} | "
        f"Meal: {df['meal'][ind]} | "
        f"Is Repeated Guest: {df['is_repeated_guest'][ind]} | "
        f"Reserved Room Type: {df['reserved_room_type'][ind]} | "
        f"Days in Waiting List: {df['days_in_waiting_list'][ind]} | "
        f"Customer Type: {df['customer_type'][ind]} | "
        f"Average_Daily_Rate: {df['adr'][ind]} | \n#####\n"
    )
    # Create LangChain Document object
    doc = Document(page_content=text, metadata={"source": "local"})
    documents.append(doc)


In [ ]:
# Print all documents
for doc in documents[:5]:
    print(doc.page_content)  # Prints the formatted text
    print("-" * 80)  # Separator for better readability


Hotel: Resort Hotel | Lead Time: 13 | Year: 2015 | Month: July | Weekend Nights: 0 | Week Nights: 1 | Country: GBR | Reservation Status: Check-Out | Reservation Status Date: 2015-02-07 | Is Canceled: 0 | Adults: 1 | Children: 0.0 | Babies: 0 | Meal: BB | Is Repeated Guest: 0 | Reserved Room Type: A | Days in Waiting List: 0 | Customer Type: Transient | Average_Daily_Rate: 75.0 | 
#####

--------------------------------------------------------------------------------
Hotel: Resort Hotel | Lead Time: 14 | Year: 2015 | Month: July | Weekend Nights: 0 | Week Nights: 2 | Country: GBR | Reservation Status: Check-Out | Reservation Status Date: 2015-03-07 | Is Canceled: 0 | Adults: 2 | Children: 0.0 | Babies: 0 | Meal: BB | Is Repeated Guest: 0 | Reserved Room Type: A | Days in Waiting List: 0 | Customer Type: Transient | Average_Daily_Rate: 98.0 | 
#####

--------------------------------------------------------------------------------
Hotel: Resort Hotel | Lead Time: 14 | Year: 2015 | Month: 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the documents into smaller chunks
#separators=["\n\n", "\n", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents(documents)

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

<ipython-input-27-6649f5995654>:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(tiny_llm, retriever,return_source_documents=True)

In [ ]:
import sys
import re

def is_calculation_query(query):
    keywords = ["total", "sum", "average", "highest", "lowest", "count", "revenue", "price", "cancellations"]
    return any(word in query.lower() for word in keywords)

def handle_calculation_query(query, df):
    if "total revenue" in query.lower():
        match = re.search(r"(\w+)\s+(\d{4})", query)  # Extracts Month & Year
        if match:
            month, year = match.groups()
            filtered_df = df[(df['arrival_date_month'] == month) & (df['arrival_date_year'] == int(year))]
            total_revenue = filtered_df['total_revenue'].sum()
            return f"Total revenue for {month} {year} is {total_revenue:.2f}"
        return "Please specify a valid month and year."

    elif "room price per night of each hotel of each month" in query.lower():
        avg_price_per_hotel_month = df.groupby(['hotel', 'arrival_date_month'])['adr'].mean().reset_index()
        result_text = "Room Price Per Night for Each Hotel (by Month):\n"
        for index, row in avg_price_per_hotel_month.iterrows():
            result_text += f"Hotel: {row['hotel']}, Month: {row['arrival_date_month']}, Price per Night: {row['adr']:.2f}\n"
        return result_text

    elif "cancellation rate" in query.lower() and "each hotel" in query.lower():
        cancellation_stats = df.groupby('hotel')['is_canceled'].agg(['sum', 'count']).reset_index()
        cancellation_stats['cancellation_rate'] = (cancellation_stats['sum'] / cancellation_stats['count']) * 100

        result_text = "Cancellation Rate for Each Hotel:\n"
        for index, row in cancellation_stats.iterrows():
            result_text += f"Hotel: {row['hotel']}, Cancellation Rate: {row['cancellation_rate']:.2f}%\n"
        return result_text

    elif "country wise guests" in query.lower():
        # Calculate total guests per country
        df['total_guests'] = df['adults'] + df.get('children', 0) + df.get('babies', 0)
        country_guests = df.groupby('country')['total_guests'].sum().sort_values(ascending=False)

        # Display table output
        print(country_guests.head(10))  # Show top 10 countries


    return "I couldn't understand the calculation request."

# Start Query Loop
chat_history = []
while True:
    query = input("Prompt (type 'exit' to quit): ")

    if query.lower() == "exit":
        print("Exiting...")
        break

    if is_calculation_query(query):
        result = handle_calculation_query(query, df)
    else:
        result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
        result = result['answer']

    print('Answer:', result, '\n')
    chat_history.append((query, result))


Prompt (type 'exit' to quit): country wise guests
country
PRT    72208.0
GBR    22366.0
FRA    20495.0
ESP    16914.0
DEU    13723.0
ITA     7508.0
IRL     6573.0
BEL     4596.0
BRA     4578.0
USA     4083.0
Name: total_guests, dtype: float64
Answer: I couldn't understand the calculation request. 

Prompt (type 'exit' to quit): cancellation rate
Answer:  The customer service team has provided the information requested. 

Prompt (type 'exit' to quit): What is the cancellation rate of hotel?
Answer:  The cancellation rate for the hotel is 1 out of 4 reservations. 

Prompt (type 'exit' to quit): what is the room price per night of each hotel of each month?
Answer: Room Price Per Night for Each Hotel (by Month):
Hotel: City Hotel, Month: April, Price per Night: 112.40
Hotel: City Hotel, Month: August, Price per Night: 116.18
Hotel: City Hotel, Month: December, Price per Night: 90.09
Hotel: City Hotel, Month: February, Price per Night: 86.99
Hotel: City Hotel, Month: January, Price per Nigh